In [6]:
import os

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
    original_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(original_dir)))))
    os.chdir(original_dir)
exec(open('setup/default.py').read())
import scipy.stats as stats


In [7]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')
menu_w_sold_out['menu_no'] = menu_w_sold_out.groupby(['day', 'meal_time'])['day'].transform('size')
menu_w_sold_out = menu_w_sold_out[(menu_w_sold_out.menu_no>1)&
                     (menu_w_sold_out.day>='2023-06-01')]

In [18]:
# 요일
target_meal = 'lunch'
lunch_df= menu_w_sold_out[(menu_w_sold_out.meal_time == target_meal)].groupby('day_of_week').agg(
    tot_day=('day', lambda x: x.nunique()),    # 고유한 date 개수 계산하여 n에 저장
    soldout_day=('day', lambda x: x[menu_w_sold_out.is_soldout==True].nunique()) # is_na=True 조건을 만족하는 고유한 date 개수 계산하여 n2에 저장
).reset_index()
lunch_df['un_soldout_day'] = lunch_df['tot_day']-lunch_df['soldout_day']
lunch_df['ratio'] = lunch_df['soldout_day']/lunch_df['tot_day']

target_meal = 'dinner'
dinner_df= menu_w_sold_out[(menu_w_sold_out.meal_time == target_meal)].groupby('day_of_week').agg(
    tot_day=('day', lambda x: x.nunique()),    # 고유한 date 개수 계산하여 n에 저장
    soldout_day=('day', lambda x: x[menu_w_sold_out.is_soldout==True].nunique()) # is_na=True 조건을 만족하는 고유한 date 개수 계산하여 n2에 저장
).reset_index()
dinner_df['un_soldout_day'] = dinner_df['tot_day']-dinner_df['soldout_day']
dinner_df['ratio'] = dinner_df['soldout_day']/dinner_df['tot_day']


In [19]:
# 코드 참고 : https://brunch.co.kr/@saetae/154
# 카이제곱 통계량 , p-value, 자유도, 기대빈도 출력 
# 점심 
# 품절이 많은 편 : 화/금, 적은편 : 수/목
chi2,p, dof, ef = stats.chi2_contingency(lunch_df[['soldout_day','un_soldout_day']])
print(chi2, p, dof)
print(lunch_df)
print(ef)

9.449056768180444 0.050804608176902355 4
  day_of_week  tot_day  soldout_day  un_soldout_day     ratio
0      Friday       43           29              14  0.674419
1      Monday       44           27              17  0.613636
2    Thursday       44           26              18  0.590909
3     Tuesday       46           32              14  0.695652
4   Wednesday       44           18              26  0.409091
[[25.68325792 17.31674208]
 [26.28054299 17.71945701]
 [26.28054299 17.71945701]
 [27.47511312 18.52488688]
 [26.28054299 17.71945701]]


In [20]:
# 품절이 많은 편 : 화, 적은편 : 월/목
chi2,p, dof, ef = stats.chi2_contingency(dinner_df[['soldout_day','un_soldout_day']])
print(chi2, p, dof)
print(dinner_df)
print(ef)

3.833487940630798 0.2800148767789154 3
  day_of_week  tot_day  soldout_day  un_soldout_day     ratio
0      Monday       44           27              17  0.613636
1    Thursday       49           30              19  0.612245
2     Tuesday       49           38              11  0.775510
3   Wednesday       44           29              15  0.659091
[[29.33333333 14.66666667]
 [32.66666667 16.33333333]
 [32.66666667 16.33333333]
 [29.33333333 14.66666667]]
